In [52]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from db_manager import SQLiteWrapper
from functions_notion import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
sqlite_path = "KoboReader.sqlite"

In [3]:
pd.options.display.max_columns = None

# Carga de BBDD

In [4]:
db_path = "KoboReader.sqlite"  # Ruta a tu archivo SQLite
db = SQLiteWrapper(db_path)
db.connect()  # Establecer conexión
anotaciones_df = db.get_annotations()
# db.close()

In [8]:
db.get_query_df("""
SELECT * --Title, PageCount
FROM Content
--WHERE PageCount IS NOT NULL;
""")

,ContentID,ContentType,MimeType,BookID,BookTitle,ImageId,Title,Attribution,Description,DateCreated,ShortCoverKey,adobe_location,Publisher,IsEncrypted,DateLastRead,FirstTimeReading,ChapterIDBookmarked,ParagraphBookmarked,BookmarkWordOffset,NumShortcovers,VolumeIndex,___NumPages,ReadStatus,___SyncTime,___UserID,PublicationId,___FileOffset,___FileSize,___PercentRead,___ExpirationStatus,FavouritesIndex,Accessibility,ContentURL,Language,BookshelfTags,IsDownloaded,FeedbackType,AverageRating,Depth,PageProgressDirection,InWishlist,ISBN,WishlistedDate,FeedbackTypeSynced,IsSocialEnabled,EpubType,Monetization,ExternalId,Series,SeriesNumber,Subtitle,WordCount,Fallback,RestOfBookEstimate,CurrentChapterEstimate,CurrentChapterProgress,PocketStatus,UnsyncedPocketChanges,ImageUrl,DateAdded,WorkId,Properties,RenditionSpread,RatingCount,ReviewsSyncDate,MediaOverlay,MediaOverlayType,RedirectPreviewUrl,PreviewFileSize,EntitlementId,CrossRevisionId,DownloadUrl,ReadStateSynced,TimesStartedReading,TimeSpentReading,LastTimeStartedReading,LastTimeFinishedReading,ApplicableSubscriptions,ExternalIds,PurchaseRevisionId,SeriesID,SeriesNumberFloat,AdobeLoanExpiration,HideFromHomePage,IsInternetArchive,titleKana,subtitleKana,seriesKana,attributionKana,publisherKana,IsPurchaseable,IsSupported,AnnotationsSyncToken,DateModified,StorePages,StoreWordCount,StoreTimeToReadLowerEstimate,StoreTimeToReadUpperEstimate,Duration,IsAbridged,SyncConflictType
0,22b09ad5-4b26-4130-9424-9cee56aea26f,6,application/x-kobo-epub+zip,None,None,68fd5c91-e759-4154-9b61-eef242623461,Something to Hide,Elizabeth George,<p><strong>Detective Sergeant Barbara Havers a...,2022-01-11T05:00:00.0000000Z,None,None,Penguin Publishing Group,false,None,true,None,0,0,0.0,0,-1,0,2022-01-15T18:53:29Z,,None,NaN,0,0,0.0,-1,-1,None,en,None,false,0,5.0,0,default,FALSE,9780593296851,None,0,true,13,2,None,A Lynley Novel,None,A Lynley Novel,-1,None,0.0,0.0,0.0,0,,None,None,d16d6eff-b841-3d04-ab2b-13b5158458e6,None,None,1,None,None,None,true,1004170.0,,04710a6e-cb96-3ee2-98d5-d11701f288bd,false,true,0.0,0.0,None,None,None,None,None,None,0.0,None,false,false,None,None,None,None,None,true,true,None,None,0,0,0,0,0,None,0
1,300e8011-45e8-45c8-a010-917c21a983d0,6,application/x-kobo-epub+zip,None,None,1f968fbd-c98a-4880-a02d-aa9cd1f811c6,End of Days,Brad Taylor,<p><strong>Pike Logan must stop a deranged kil...,2022-01-11T05:00:00.0000000Z,None,None,HarperCollins,false,None,true,None,0,0,0.0,1,-1,0,2022-01-15T18:53:29Z,,None,NaN,0,0,0.0,-1,-1,None,en,None,false,0,4.0,0,default,FALSE,9780062886125,None,0,true,13,2,None,Pike Logan,16,A Pike Logan Novel,-1,None,0.0,0.0,0.0,0,,None,None,b8b4d66b-0179-356e-bb21-70c7aa6d2e29,None,None,1,None,None,None,true,871774.0,,33549b17-771a-326e-b19d-67520108fa36,false,true,0.0,0.0,None,None,None,None,None,None,16.0,None,false,false,None,None,None,None,None,true,true,None,None,0,0,0,0,0,None,0
2,6963c490-b680-4e56-aeca-32d8c0dbd50e,6,application/x-kobo-epub+zip,None,None,0c749bd4-38ac-41c5-9d86-0c162c09e4f4,Wahala,Nikki May,"<p><strong>""Utterly winning…brings to mind Sex...",2022-01-11T05:00:00.0000000Z,None,None,HarperCollins,false,None,true,None,0,0,0.0,2,-1,0,2022-01-15T18:53:29Z,,None,NaN,0,0,0.0,-1,-1,None,en,None,false,0,0.0,0,default,FALSE,9780063084261,None,0,true,13,2,None,None,None,A Novel,-1,None,0.0,0.0,0.0,0,,None,None,4b787b13-ab87-4425-9cf3-0b348a32074b,None,None,0,None,None,None,true,769141.0,,17d18239-7275-3c4f-a98a-05a02328d001,false,true,0.0,0.0,None,None,None,None,None,None,0.0,None,false,false,None,None,None,None,None,true,true,None,None,0,0,0,0,0,None,0
3,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Sinopsis,None,None,None,None,OEBPS/Text/part0002.html,None,false,None,true,None,0,0,NaN,0,-1,0,2022-01-15T19:28:05Z,adobe_user,None,0.0,0,0,NaN,-1,-1,None,None,None,1,0,0.0,1,None,FALSE,None,None,0,true,-1,2,None,None,None,None,-1,None,NaN,NaN,NaN,0,None,None,None,Non

In [8]:
bookmark_df = db.get_query_df("""select * from bookmark""")

bookmark_flow = bookmark_df\
    .loc[lambda x: x.Text.notna()]\
    .loc[lambda x: x.VolumeID == 'file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_Mihaly_Csikszentmihalyi.epub']

In [52]:
ejemplo_pags = anotaciones_df.loc[lambda x: x.Título.str.contains("Fluir")]\
    .merge(bookmark_flow.rename(columns = {'ChapterProgress': 'Progreso del libro'}))\
    .assign(
        fragment=lambda x: x['StartContainerPath'].str.split('#').str[1],
        point=lambda x: x['fragment'].str.extract(r'point\((.*?)\)')[0],
        point_parts=lambda x: x['point'].str.split('/').apply(lambda lst: lst if lst else []),
        third_point=lambda x: x['point_parts'].apply(lambda parts: parts[3] if parts else None),
        last_point=lambda x: x['point_parts'].apply(lambda parts: parts[-1] if parts else None),
).loc[lambda x: x.third_point != x.last_point]\
.assign(annotation_point = lambda x: x.last_point.str.split(":").str[1].astype(int))\
.sort_values(["Progreso del libro", "third_point", "annotation_point"], ascending=True)\
.assign(chapter_num = lambda x: x['Capítulo'].str.extract(r"(\d+).*"))\
.loc[lambda x: x.Capítulo != "Notas"]\
[[ "chapter_num", "Progreso del libro", "third_point", "annotation_point", "Text"]]\
.drop_duplicates().head(10)


In [53]:
ejemplo_pags.assign(pag = [12, 15, 16, 17, 29, 30, 31, 31, 31, 31])\
    .assign(coc = lambda x: (x.pag/x['Progreso del libro']).round().astype(int))

,chapter_num,Progreso del libro,third_point,annotation_point,Text,pag,coc
147,1,0.034985,28,264,Los mejores momentos suelen suceder cuando el ...,12,343
52,1,0.043732,44,643,Lo que satisfaría realmente a las personas no ...,15,343
141,1,0.046647,64,17,Cuando esto se consigue y la persona se siente...,16,343
170,1,0.049563,68,878,Esta insatisfacción crónica es el segundo obst...,17,343
162,1,0.084548,154,166,La única autoridad en la que creen muchas pers...,29,343
181,1,0.087464,164,292,La solución es liberarse de forma gradual de l...,30,343
81,1,0.090379,166,86,encontrar recompensas en los acontecimientos d...,31,343
84,1,0.090379,166,1256,"inclinaciones biológicas, estaremos controlado...",31,343
83,1,0.090379,166,1778,"«Los hombres no tienen miedo de las cosas, sin...",31,343
82,1,0.090379,166,1927,"Si te sientes dolido por las cosas externas, n...",31,343


In [ ]:

#.Text.values[1]
# .assign(pagina = lambda x: [12])
# .astype(float)\
# .sort_values("Progreso del libro").tail(20)
# .plot(x = "Progreso del libro", y = "third_point", kind = "scatter")

In [ ]:
12, 15, 16, 17, 29, 30, 31, 31, 31 

## Exploratorio paginas libros

In [60]:
content = db.get_query_df("""SELECT * from content""")

In [76]:
content.astype(str).applymap(lambda x: "Fluir" in x).sum().loc[lambda x: x>0]

C:\Users\jaguirrepeman\AppData\Local\Temp\ipykernel_22836\633017987.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  content.astype(str).applymap(lambda x: "Fluir" in x).sum().loc[lambda x: x>0]


ContentID    17
BookID       16
BookTitle    16
ImageId       1
Title         2
dtype: int64

In [131]:
content_fluir = content.astype(str).loc[lambda x: x.ContentID.str.contains("Fluir")]

In [ ]:
set(content_fluir.BookID.values)

{'None',
 'file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_Mihaly_Csikszentmihalyi.epub'}

In [ ]:
set(content_fluir.BookTitle.values)

{'Fluir (Flow)', 'None'}

In [116]:
content_fluir_const = content_fluir.loc[lambda x: x.ContentType == "9"].nunique().loc[lambda x: x<=1].index
content_fluir = content_fluir.drop(content_fluir_const, axis =1)

In [119]:
volume_shortcovers_df = db.get_query_df("""select * from volume_shortcovers""")

In [120]:
volume_shortcovers_fluir = volume_shortcovers_df\
    .loc[lambda x: x.volumeId.str.contains("Fluir")]

In [121]:
set(volume_shortcovers_fluir.volumeId.values)

{'file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_Mihaly_Csikszentmihalyi.epub'}

In [142]:
volume_shortcovers_fluir#.shortcoverId.values

,volumeId,shortcoverId,VolumeIndex
4799,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,0
4800,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,1
4801,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,2
4802,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,3
4803,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,4
4804,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,5
4805,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,6
4806,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,7
4807,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,8
4808,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,9


In [5]:
tables_info_df = db.get_tables_info() 

In [6]:
content

NameError: name 'content' is not defined

In [139]:
db.get_query_df("""select * from Event where ContentID like '%Fluir%'""")

,EventType,FirstOccurrence,LastOccurrence,EventCount,ContentID,ExtraData,Checksum
0,1012,None,2024-11-16T22:56:11.000,1,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,        e v e n t T i m e s t a m p s   \t   ...,8227810199b3ad7a4f28b32ec11dba76
1,8,None,2024-11-22T22:55:45.000,2,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,        e v e n t T i m e s t a m p s   \t   ...,5e484c56246cbd539fae55f6b48a46b2
2,1013,None,2024-12-03T06:28:54.000,2,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,        e v e n t T i m e s t a m p s   \t   ...,c455d83d06150f4315e7d73a24efd2ec
3,1014,None,2024-12-10T06:07:28.000,2,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,        e v e n t T i m e s t a m p s   \t   ...,c7c153f2f232078bbac04bad4cda7bc4
4,46,2024-12-17T05:36:49.000,2024-12-17T05:36:49.000,1032,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,   \r    w o r d s R e a d            w o r...,edd7442697cebcc904904b6f739a1525
5,3,None,2024-12-17T05:36:49.000,52,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,   \r    w o r d s R e a d            w o r...,1d47ef108461997570fccd8f5dddad98
6,1020,None,2024-12-17T05:36:49.000,52,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,        e v e n t T i m e s t a m p s   \t   ...,5f589e197f6f88d62902300bef4fd664
7,9,None,2024-12-17T05:51:52.000,221,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,        e v e n t T i m e s t a m p s   \t   ...,cc1dd9a2f38c303cb4d61a69b69f6c62
8,1021,None,2024-12-17T05:54:41.000,51,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,        e v e n t T i m e s t a m p s   \t   ...,dd92a25dd13c135911cb0eff726248c4
9,5,None,2024-12-17T05:54:41.000,1,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,        e v e n t T i m e s t a m p s   \t   ...,6cd71a8fbbf0c5f22e8eb287008b97ba


In [156]:
content.loc[lambda x:( x.ContentType == "6")&(x.___FileSize > 0)]\
    [['Title', 'Attribution', '___FileSize', 'ISBN']]\
    .sort_values("___FileSize", ascending=False)

,Title,Attribution,___FileSize,ISBN
3278,How to Win at Chess,Levy Rozman,66619326,9781984862082
6078,The Storyteller,Dave Grohl,49450772,9780063076112
6231,Thing Explainer,Randall Munroe,31550238,urn:uuid:1E0D10B9-143C-45C7-9AB5-D7B75374C8F6
5925,SPQR,Mary Beard,27436002,urn:uuid:3ce9081f-7d86-4ed5-ac1b-62f39528d28e
3991,Más allá del orden,Jordan Peterson,7452163,urn:uuid:7758ef22-8fe5-445e-8acc-2f0c0232cc2f
...,...,...,...,...
2587,CAPÍTULO 26,None,13010,9908b286-3de6-4a66-ad70-fff17f5544f0
2594,CAPITULO 30,None,12533,4fee2156-b8df-4427-9e4b-8614b87d5b6c
2585,CAPÍTULO 25,None,12006,38b30760-eaea-4cf8-b315-1ce0ead21e95
2344,CAPITULO 18,None,11931,e9aa3cb4-10e8-4965-a26d-eb0a99b9dfb6


In [ ]:
file_size: 922260
isbn: urn:uuid:53262b0f-753f-4cec-babf-5618ca617244
work_id: 6218eba89c0118c86dc51.ccff5cfa622945a674febd26
DateLastRead: 

## Exploratorio libros

In [55]:
libros_df = db.get_query_df("""SELECT Title, Attribution, Description, ___SyncTime, DateCreated, DateLastRead, 
                               NumShortcovers, ReadStatus, ___PercentRead, Language, ReadStateSynced, TimeSpentReading 
                               FROM content
                               WHERE ContentType = "6" AND EpubType = -1 AND ContentID LIKE 'file%'""")

In [59]:
libros_df.loc[lambda x: x.Title.str.contains("Flu|amar")]

,Title,Attribution,Description,___SyncTime,DateCreated,DateLastRead,NumShortcovers,ReadStatus,___PercentRead,Language,ReadStateSynced,TimeSpentReading
124,Fluir (Flow),Mihály Csíkszentmihályi,Cada año se publican en el mundo multitud de t...,2024-09-08T12:43:46Z,None,2024-12-17T05:54:41Z,16,2,99,es,false,35003
155,El arte de amar,Erich Fromm,«El arte de amar» es una obra con la que Erich...,2024-08-27T23:35:43Z,None,2025-01-12T16:47:02Z,14,2,99,es,false,20600


In [56]:
libros_df.sort_values("___SyncTime", ascending=True).head(10)

,Title,Attribution,Description,___SyncTime,DateCreated,DateLastRead,NumShortcovers,ReadStatus,___PercentRead,Language,ReadStateSynced,TimeSpentReading
59,El juego del alma,Javier Castillo,"&lt;b&gt;Nueva York, 2011&lt;/b&gt;. Una chica...",2022-01-15T19:28:05Z,None,2024-08-07T21:54:51.374,54,2,100,es,false,27041
7,A fuego lento,Paula Hawkins,Con la misma intensidad con la que ha cautivad...,2022-01-15T19:28:05Z,None,2022-07-16T09:18:00Z,63,2,99,es,false,18417
3,La bestia,Carmen Mola,"<div>\n<p>Corre el año 1834 y Madrid, una pequ...",2022-01-15T19:28:05Z,None,2022-03-02T17:13:08Z,95,2,99,es-ES,false,31637
1,Soy introvertido. ¿Y qué?,Steve Allen,Es duro ser un introvertido en un mundo extrov...,2022-02-02T22:46:04Z,None,2022-02-26T14:24:37Z,54,1,40,es,false,3161
0,Scar Tissue,Anthony Kiedis,<p><cite>Scar Tissue</cite> es la autobiografí...,2022-02-03T19:18:11Z,None,2022-02-19T19:47:04Z,21,2,99,es,false,35693
2,OPEN,Andre Agassi,"Siendo un bebé, le pusieron una raqueta de jug...",2022-02-15T20:11:19Z,None,2022-02-28T21:06:07Z,36,2,99,es,false,36955
5,El guardián invisible,Dolores Redondo,"En los márgenes del río Baztán, en el valle de...",2022-03-05T11:21:17Z,None,2022-03-16T21:06:58Z,49,2,99,es,false,37359
4,El lobo de Wall Street,Jordan Belfort,«La historia real en la que se basa la películ...,2022-03-05T14:21:36Z,None,None,51,0,0,es,true,0
37,RAFA: Mi historia,Rafael Nadal,La hazaña de Rafael Nadal de convertirse en el...,2022-03-05T14:21:53Z,None,2023-12-30T22:44:55Z,23,2,99,es,false,25395
6,Legado en los huesos,Dolores Redondo,<p>El juicio contra el padrastro de la joven J...,2022-03-18T16:37:45Z,None,2022-03-27T10:14:56Z,50,2,99,es,false,45058


In [124]:
tables_info_df = db.get_tables_info() 

In [125]:
tables_info_df.sort_values("num_filas", ascending= False).head(10)

,nombre,num_filas,num_cols,col_names
1,content,6720,101,"[ContentID, ContentType, MimeType, BookID, Boo..."
3,volume_shortcovers,6540,3,"[volumeId, shortcoverId, VolumeIndex]"
5,Bookmark,1201,24,"[BookmarkID, VolumeID, ContentID, StartContain..."
6,Event,1093,7,"[EventType, FirstOccurrence, LastOccurrence, E..."
31,KoboPlusAssets,294,5,"[AssetGroupId, Key, Language, Type, Value]"
29,DropboxItem,154,2,"[Id, Json]"
18,Activity,118,6,"[Id, Enabled, Type, Action, Date, Data]"
26,WordList,83,4,"[Text, VolumeId, DictSuffix, DateCreated]"
11,ShelfContent,66,5,"[ShelfName, ContentId, DateModified, _IsDelete..."
8,Rules,33,10,"[AchievementId, EventProperty, EventType, Goal..."


In [7]:
content_df = db.get_query_df("""select * from content""")
const_cols_content = content_df.nunique().loc[lambda x: x<=1].index
content_df = content_df.drop(const_cols_content, axis = 1)

In [21]:
content_df.loc[lambda x: x.ContentID.str.contains("A_fuego_lento")]

,ContentID,ContentType,MimeType,BookID,BookTitle,ImageId,Title,Attribution,Description,DateCreated,adobe_location,Publisher,DateLastRead,FirstTimeReading,ChapterIDBookmarked,NumShortcovers,VolumeIndex,___NumPages,ReadStatus,___SyncTime,___UserID,___FileSize,___PercentRead,Language,IsDownloaded,AverageRating,Depth,PageProgressDirection,ISBN,EpubType,Series,SeriesNumber,Subtitle,WorkId,RatingCount,RedirectPreviewUrl,PreviewFileSize,CrossRevisionId,ReadStateSynced,TimeSpentReading,SeriesNumberFloat,IsInternetArchive,Duration
3,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Sinopsis,None,None,None,OEBPS/Text/part0002.html,None,None,true,None,NaN,0,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
4,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Portadilla,None,None,None,OEBPS/Text/part0003.html,None,None,true,None,NaN,1,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
5,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,La expresión inglesa...,None,None,None,OEBPS/Text/part0004.html,None,None,true,None,NaN,2,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
6,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Dedicatoria,None,None,None,OEBPS/Text/part0005.html,None,None,true,None,NaN,3,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
7,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Mapa,None,None,None,OEBPS/Text/part0006.html,None,None,true,None,NaN,4,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Agradecimientos,None,None,None,OEBPS/Text/part0062.html,None,None,true,None,NaN,59,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
63,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Permisos de reproducción,None,None,None,OEBPS/Text/part0063.html,None,None,true,None,NaN,60,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
64,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Créditos,None,None,None,OEBPS/Text/part0064.html,None,None,true,None,NaN,61,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
65,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,¡Encuentra aquí tu próxima lectura!,None,None,None,OEBPS/Text/part0065.html,None,None,true,None,NaN,62,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0


In [16]:
volume_shortcovers_df = db.get_query_df("""select * from volume_shortcovers""")

In [20]:
volume_shortcovers_df.shortcoverId.values[:5]

array(['file:///mnt/onboard/A_fuego_lento_Paula_Hawkins.epub#(0)OEBPS/Text/part0002.html',
       'file:///mnt/onboard/A_fuego_lento_Paula_Hawkins.epub#(1)OEBPS/Text/part0003.html',
       'file:///mnt/onboard/A_fuego_lento_Paula_Hawkins.epub#(2)OEBPS/Text/part0004.html',
       'file:///mnt/onboard/A_fuego_lento_Paula_Hawkins.epub#(3)OEBPS/Text/part0005.html',
       'file:///mnt/onboard/A_fuego_lento_Paula_Hawkins.epub#(4)OEBPS/Text/part0006.html'],
      dtype=object)

In [12]:
content_df.loc[lambda x: x.ContentID.str.contains("^file")].ContentID.unique()

array(['file:///mnt/onboard/A_fuego_lento_Paula_Hawkins.epub#(0)OEBPS/Text/part0002.html',
       'file:///mnt/onboard/A_fuego_lento_Paula_Hawkins.epub#(1)OEBPS/Text/part0003.html',
       'file:///mnt/onboard/A_fuego_lento_Paula_Hawkins.epub#(2)OEBPS/Text/part0004.html',
       ...,
       'file:///mnt/onboard/.kobo/dropbox/El_arte_de_amar_Erich_Fromm (1).epub',
       'file:///mnt/onboard/.kobo/dropbox/Amanecer_Conn_Iggulden.epub',
       'file:///mnt/onboard/.kobo/dropbox/Las 5 trampas del amor -- Iñaki Piñuel [Piñuel, Iñaki] -- 2017 -- La Esfera -- 9788491640158 -- 83c3b5397c8535db5ddd231a3ed0721e -- Anna’s Archive.epub'],
      dtype=object)

In [13]:
content_df.loc[lambda x: x.ContentID.str.contains('file:///mnt/onboard/A_fuego_lento_Paula_Hawkins')]

,ContentID,ContentType,MimeType,BookID,BookTitle,ImageId,Title,Attribution,Description,DateCreated,adobe_location,Publisher,DateLastRead,FirstTimeReading,ChapterIDBookmarked,NumShortcovers,VolumeIndex,___NumPages,ReadStatus,___SyncTime,___UserID,___FileSize,___PercentRead,Language,IsDownloaded,AverageRating,Depth,PageProgressDirection,ISBN,EpubType,Series,SeriesNumber,Subtitle,WorkId,RatingCount,RedirectPreviewUrl,PreviewFileSize,CrossRevisionId,ReadStateSynced,TimeSpentReading,SeriesNumberFloat,IsInternetArchive,Duration
3,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Sinopsis,None,None,None,OEBPS/Text/part0002.html,None,None,true,None,NaN,0,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
4,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Portadilla,None,None,None,OEBPS/Text/part0003.html,None,None,true,None,NaN,1,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
5,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,La expresión inglesa...,None,None,None,OEBPS/Text/part0004.html,None,None,true,None,NaN,2,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
6,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Dedicatoria,None,None,None,OEBPS/Text/part0005.html,None,None,true,None,NaN,3,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
7,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Mapa,None,None,None,OEBPS/Text/part0006.html,None,None,true,None,NaN,4,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Agradecimientos,None,None,None,OEBPS/Text/part0062.html,None,None,true,None,NaN,59,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
63,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Permisos de reproducción,None,None,None,OEBPS/Text/part0063.html,None,None,true,None,NaN,60,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
64,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Créditos,None,None,None,OEBPS/Text/part0064.html,None,None,true,None,NaN,61,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
65,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,¡Encuentra aquí tu próxima lectura!,None,None,None,OEBPS/Text/part0065.html,None,None,true,None,NaN,62,-1,0,2022-01-15T19:28:05Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0


In [14]:
bookmark_df = db.get_query_df("""select * from bookmark""")

In [15]:
bookmark_df\
    .loc[lambda x: x.Text.notna()]\
    .loc[lambda x: x.VolumeID == 'file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_Mihaly_Csikszentmihalyi.epub']\
    [['StartContainerPath', 'ChapterProgress']]

,StartContainerPath,ChapterProgress
462,OEBPS/Text/02.xhtml#point(/1/4/14/1:54),0.104956
463,OEBPS/Text/08.xhtml#point(/1/4/84/1:489),0.580175
464,OEBPS/Text/08.xhtml#point(/1/4/112/1:116),0.597668
465,OEBPS/Text/09.xhtml#point(/1/4/90/1/1:3),0.676385
466,OEBPS/Text/08.xhtml#point(/1/4/115:4),0.597668
...,...,...
636,OEBPS/Text/notas.xhtml#point(/1/4/4/748/2:160),0.985423
637,OEBPS/Text/notas.xhtml#point(/1/4/4/748/2:982),0.985423
638,OEBPS/Text/notas.xhtml#point(/1/4/4/770/2:229),0.988338
639,OEBPS/Text/notas.xhtml#point(/1/4/4/770/2:546),0.988338


In [75]:
bookmark_df.loc[lambda x: x.Text.notna()].groupby("VolumeID").size().sort_values(ascending = False)

VolumeID
file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_Mihaly_Csikszentmihalyi.epub                                                                                                                                                                                              178
file:///mnt/onboard/.kobo/dropbox/Surrounded by idiots_ the four types of human behavior and -- Bradbury, Rod;Erikson, Thomas;Pender, Martin -- St_ Martin's Essentials, First Edition, -- 9781250179944 -- d5276a97a2db0fa3010cac81bdeb6ac3 -- Anna’s Archive.epub    158
file:///mnt/onboard/.kobo/dropbox/El_arte_de_amar_Erich_Fromm (1).epub                                                                                                                                                                                                 144
file:///mnt/onboard/.kobo/dropbox/Los dones de la imperfección_ Líbrate de quien crees que -- Brené Brown -- 2014 -- Penguin Random House Grupo Editorial USA -- 9781941999004 -- 95fd718adb21

# Exportacion a Notion

In [7]:
from notion_client import Client
from config import NOTION_API_TOKEN, NOTION_BOOKS_DATABASE_ID, NOTION_ANNOTATIONS_DATABASE_ID

# Configuración inicial
notion = Client(auth=NOTION_API_TOKEN)


In [6]:
# create_books(anotaciones_df, notion, NOTION_BOOKS_DATABASE_ID)

In [7]:
# create_annotations(anotaciones_df, notion, NOTION_ANNOTATIONS_DATABASE_ID, NOTION_BOOKS_DATABASE_ID)

In [9]:
create_book_pages(anotaciones_df, notion, NOTION_BOOKS_DATABASE_ID, clear_content = True)

Contenido existente eliminado para el libro '12 reglas para vivir'.
Contenido existente eliminado para el libro 'Acontece que no es poco'.
Contenido existente eliminado para el libro 'Amanecer'.
Contenido existente eliminado para el libro 'Breaking the Habit of Being Yourself'.
Contenido existente eliminado para el libro 'Como hacer que te pasen cosas buenas'.
Contenido existente eliminado para el libro 'El arte de amar'.
Contenido existente eliminado para el libro 'El hombre en busca de sentido'.
Contenido existente eliminado para el libro 'El juego del alma'.
Contenido existente eliminado para el libro 'El poder de confiar en ti mismo'.
Contenido existente eliminado para el libro 'El universo de lo sencillo'.
Contenido existente eliminado para el libro 'Encuentra tu persona vitamina'.
Contenido existente eliminado para el libro 'Fluir (Flow)'.
Contenido existente eliminado para el libro 'Google Cloud Certified Professional Cloud Architect Study Guide'.
Contenido existente eliminado p

# Dropbox

In [ ]:
import dropbox
from epub_processor import EpubProcessor
# Conecta con tu cuenta usando el Access Token
ACCESS_TOKEN = 'sl.u.AFgxr6adK-kjWCiqfhIngQKaVJjMlERUK1XDIfRybN8r5zOx7Xni_g_87-kVw2Z8o-v5PQfzrzk-jloSq1byM3EF_ztCN8I5dZGlCL5vu0J8ufjwlWTTbhJaKYreASWxcmvPnMO97DO-Rhg_gueANLAYU3Ko_VKHuSTkb0w2Uu_PR-JjMyJg7XyuiWWFrfDoDS8d0dfZLje3XyDeZrCWgXRLZJ7YHRodu9JLLl4G27KXD0KuLtOe5I4u08M9YJK5laBCgEP7Sik29IEu9qkQwgNwsPSzzaw4xnLnxyP44mnAn1XG8zR7EozNJNQJINWqzcNufKQwNa46R98IJ7VoyX7bPAkuADtr26wfLQVqeyHsOSHZfVKFyi72d5BfN2lw9cNpjAGToS9Ov9aVVwHGPw3u55RCSEBFVj8BFIs8RbnBVFonykDhR2eVn_OuMnMompHTP2vzHgtyN0UkAPl0i0jMIExOhRf2VSZ1ilvPofIthVZy-WH1gj0Bs9d0Y388fQmW9TGHp9sfRUPPUuN-4lMChsTjxfIP3SKK05PexaytmF7ZDF67rPY9EAXSFsNHP01m53pfG0QB5gFzJwTa6og3hEZoaXOCvKu5s21TcRuLdEwckFTsqvWHvieSczkSIeasIdoO4x5GZtPf2b3agLKK8B8yVevaWd1ERAC46r3DpwLmKJ14L_sVxwXqBzAKclBsUydUaU-wT-bosqpbCwGVOfVjOrJr5zLM1q0v76Jvf6kTaUvo10TWF91m73nDw6DE7gUBVAmPehEmtBxP-2fgU-kw-x-Ea1e6ZH3JjgH9RVKlWGjVfHGVb4qwU6WIX1G6aBddGMbAs5jZlBbh0q8XWniVWb7HGHeBy3WKxYbi0YomLQM5N0tcOjZMRk7kU8ykY1QWkmq8LG3aERvnZ2npL6vMV-v9Y6ZEZYysKSOr9ECyroEJvwZp_DJt4BwKSgqMar4tXT27RAT_GzB2kHueV-a_gXqgA51cJx3BU1AgcMtT0XstIRReSCBNAZBOPKtidqvfR8sD8BHqz-LDzugDtM1ZOps9c8R-G4iYv1j-dpDEPZSNJT5_ixvJobw8CWthOpV1xR7Qo0AjRjhUqqrXCkB6asWIXBaDXIJMGLV1QctlC9n9vHh5kEcLypO1_RdZdzX1upo6XYwU8PX3k8QXjxYsF6akLTauKBlKYiIkyvfuw3nrNseClDSwFARd0agNQxTJTupOsUpQg7ZXMs9ZuFuMrQZb2WmX8V6zUFOxx-BClzG_SCuO3JgIBErcceEs9bmYc27N-KDBo0B6pk2IObulEl_3YV9HYcV7OucGOw'


In [ ]:
import dropbox
import pandas as pd
import json

# Función que conecta con Dropbox, procesa los archivos EPUB y devuelve un DataFrame con los metadatos
def get_epub_metadata_from_dropbox(ACCESS_TOKEN, folder_path='/Aplicaciones/Rakuten Kobo'):
    # Lista para almacenar los metadatos de todos los libros
    all_metadata = []

    try:
        # Conexión a Dropbox
        dbx = dropbox.Dropbox(ACCESS_TOKEN)
        response = dbx.files_list_folder(path=folder_path)

        print("Archivos en la carpeta Rakuten Kobo:")
        for entry in response.entries:
            if entry.name.endswith('.epub'):
                try:
                    print(f"Procesando archivo: {entry.name}")

                    # Descargar el archivo .epub directamente en memoria
                    metadata, res = dbx.files_download(path=f"{folder_path}/{entry.name}")
                    epub_content = res.content  # Asegúrate de obtener los bytes del contenido

                    # Usar la clase EpubProcessor para procesar el archivo directamente desde la memoria
                    processor = EpubProcessor(epub_content=epub_content)
                    processor.process()

                    # Obtener los metadatos del archivo EPUB
                    metadata = processor.get_metadata()
                    metadata['filename'] = entry.name  # Agregar el nombre del archivo como columna

                    # Agregar los metadatos a la lista
                    all_metadata.append(metadata)
                except:
                    print(f"Fallo en {entry.name}")
        # Crear un DataFrame con todos los metadatos
        df = pd.DataFrame(all_metadata)

        return df

    except dropbox.exceptions.AuthError as e:
        print(f"Error de autenticación: {e}")
    except dropbox.exceptions.ApiError as e:
        print(f"Error de la API: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")

# Ejemplo de uso
df_epub_metadata = get_epub_metadata_from_dropbox(ACCESS_TOKEN)

# Mostrar el DataFrame resultante
print(df_epub_metadata)


In [ ]:
def calculate_pages_kobo_style(epub_content, chars_per_page=1024):
    """DEPRECATED"""
    # Extraer contenido del .epub (contenido en formato zip)
    text = ""
    with zipfile.ZipFile(epub_content) as epub:
        for file in epub.namelist():
            if file.endswith('.html') or file.endswith('.xhtml'):
                with epub.open(file) as f:
                    content = f.read().decode('utf-8')
                    # Eliminar etiquetas HTML y quedarnos con el texto
                    text += re.sub(r'<[^>]+>', '', content)
    
    # Calcular el número de páginas basado en caracteres
    num_pages = len(text) // chars_per_page
    return num_pages

# Exploratorio tablas interesantes

In [6]:
QUERY_ITEMS = """
    SELECT 
        b.VolumeID, 
        b.Text, 
        b.Annotation, 
        b.DateCreated, 
        b.DateModified,
        b.ChapterProgress, 
        b.Type as Type,
        c.BookTitle, 
        c.Title as Chapter, 
        c.Attribution as Author, 
        b.BookmarkID
    FROM Bookmark b INNER JOIN content c
    ON b.ContentID = c.ContentID 
    GROUP BY b.DateCreated 
    ORDER BY b.ChapterProgress ASC, b.DateCreated ASC;
"""

QUERY_ITEMS2 = """
    SELECT 
        b.VolumeID, 
        b.Text, 
        b.Annotation, 
        b.DateCreated, 
        b.DateModified,
        b.ChapterProgress, 
        b.Type as Type,
        c.BookTitle, 
        c.Title as Chapter, 
        c.Attribution as Author, 
        b.BookmarkID
    FROM Bookmark b INNER JOIN content c
    ON b.VolumeID = c.ContentID 
    GROUP BY b.DateCreated 
    ORDER BY b.ChapterProgress ASC, b.DateCreated ASC;
"""


QUERY_BOOKS = """
    SELECT DISTINCT
        b.VolumeID,
        c.BookTitle, 
        c.Title,
        c.Attribution as Author,
        (SELECT COUNT(*) FROM Bookmark b2 WHERE b2.VolumeID = b.VolumeID) AS Items
    FROM
        Bookmark b INNER JOIN content c ON b.VolumeID = c.ContentID
    ORDER BY
        c.Title;
"""

In [37]:
content_df.astype(str).apply(lambda x: x.str.contains("arte de amar"))\
    .sum().loc[lambda x: x>0]

BookTitle      14
Title           2
Description     1
dtype: int64

In [44]:
content_df\
    .loc[lambda x: x.BookTitle.notna()]\
    .loc[lambda x: x.BookTitle.str.contains("arte de amar")]

,ContentID,ContentType,MimeType,BookID,BookTitle,ImageId,Title,Attribution,Description,DateCreated,adobe_location,Publisher,DateLastRead,FirstTimeReading,ChapterIDBookmarked,NumShortcovers,VolumeIndex,___NumPages,ReadStatus,___SyncTime,___UserID,___FileSize,___PercentRead,Language,IsDownloaded,AverageRating,Depth,PageProgressDirection,ISBN,EpubType,Series,SeriesNumber,Subtitle,WorkId,RatingCount,RedirectPreviewUrl,PreviewFileSize,CrossRevisionId,ReadStateSynced,TimeSpentReading,SeriesNumberFloat,IsInternetArchive,Duration
3957,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,El arte de amar,None,Cubierta,None,None,None,OEBPS/Text/cubierta.xhtml,None,None,true,None,NaN,0,-1,0,2024-08-27T23:35:43Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
3958,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,El arte de amar,None,El arte de amar,None,None,None,OEBPS/Text/titulo.xhtml,None,None,true,None,NaN,1,-1,0,2024-08-27T23:35:43Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
3959,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,El arte de amar,None,Prefacio,None,None,None,OEBPS/Text/01_prefacio.xhtml,None,None,true,None,NaN,2,-1,0,2024-08-27T23:35:43Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
3960,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,El arte de amar,None,Capítulo 1: ¿Es el amor un arte?,None,None,None,OEBPS/Text/03_capitulo_1.xhtml,None,None,true,None,NaN,3,-1,0,2024-08-27T23:35:43Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
3961,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,El arte de amar,None,Capítulo 2: La teoría del amor,None,None,None,OEBPS/Text/04_capitulo_2.xhtml,None,None,true,None,NaN,4,-1,0,2024-08-27T23:35:43Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
3962,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,El arte de amar,None,"El amor, la respuesta al problema de la existe...",None,None,None,OEBPS/Text/04_capitulo_2.xhtml#sigil_toc_id_4,None,None,true,None,NaN,5,-1,0,2024-08-27T23:35:43Z,adobe_user,0,0,None,1,0.0,2,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
3963,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,El arte de amar,None,El amor entre padres e hijos,None,None,None,OEBPS/Text/04_capitulo_2.xhtml#sigil_toc_id_2,None,None,true,None,NaN,6,-1,0,2024-08-27T23:35:43Z,adobe_user,0,0,None,1,0.0,2,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
3964,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,El arte de amar,None,Los objetos amorosos,None,None,None,OEBPS/Text/04_capitulo_2.xhtml#sigil_toc_id_3,None,None,true,None,NaN,7,-1,0,2024-08-27T23:35:43Z,adobe_user,0,0,None,1,0.0,2,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
3965,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,El arte de amar,None,Capítulo 3: El amor y su desintegración en la ...,None,None,None,OEBPS/Text/05_capitulo_3.xhtml,None,None,true,None,NaN,8,-1,0,2024-08-27T23:35:43Z,adobe_user,0,0,None,1,0.0,1,None,None,-1,None,None,None,None,0,None,NaN,None,false,NaN,NaN,FALSE,0
3966,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,9,application/epub+zip

In [60]:
anotaciones_clean_df = anotaciones_df\
    .loc[lambda x: x.Type.isin(["highlight", "note"])]\
    [['BookTitle', 'VolumeID', 'Chapter', 'ChapterProgress', 'Text', 'Annotation', 'Type', 'DateCreated']]

In [65]:
anotaciones_clean_df.head(1)

,BookTitle,VolumeID,Chapter,ChapterProgress,Text,Annotation,Type,DateCreated
0,Juana de Arco,file:///mnt/onboard/.kobo/dropbox/Juana_de_Arc...,Presentación,0.019093,dos brazas.\n,None,highlight,2024-08-25T14:14:01.000


In [67]:
libros_df.drop(["BookTitle", "Items"], axis = 1)

,VolumeID,Title,Author
0,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,12 reglas para vivir,Jordan Peterson
1,file:///mnt/onboard/.kobo/dropbox/30_dias_Camb...,"30 días. Cambia de hábitos, cambia de vida",Marc Reklau
2,file:///mnt/onboard/.kobo/dropbox/Acontece_que...,Acontece que no es poco,Nieves Concostrina
3,file:///mnt/onboard/.kobo/dropbox/Amanecer_Con...,Amanecer,Conn Iggulden
4,file:///mnt/onboard/.kobo/dropbox/Breaking_The...,Breaking the Habit of Being Yourself,Joe Dispenza
5,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,Cicatriz,Juan Gómez-Jurado
6,file:///mnt/onboard/.kobo/dropbox/Como_hacer_q...,Como hacer que te pasen cosas buenas,Marian Rojas Estapé
7,file:///mnt/onboard/.kobo/dropbox/Como_ser_un_...,Cómo ser un estoico,Massimo Pigliucci
8,file:///mnt/onboard/.kobo/dropbox/Diez_negrito...,Diez negritos,Agatha Christie
9,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,El arte de amar,Erich Fromm


In [7]:
content_df = db.get_query_df("""select * from content""")
nunique_cols_content = db.get_query_df("select * from content").nunique().loc[lambda x: x<= 1].index
content_df = content_df.drop(nunique_cols_content, axis = 1)
bookmark_df = db.get_query_df("""select * from Bookmark""")
nunique_cols_bookmark = db.get_query_df("select * from Bookmark").nunique().loc[lambda x: x<= 1].index
bookmark_df = bookmark_df.drop(nunique_cols_bookmark, axis = 1)

In [8]:
anotaciones_df = db.get_query_df(QUERY_ITEMS)
anotaciones_df2 = db.get_query_df(QUERY_ITEMS2)

In [9]:
anotaciones_df.isna().sum().loc[lambda x: x>0]

Text              76
Annotation      1105
DateModified      95
Author          1201
dtype: int64

In [10]:
anotaciones_df2.isna().sum().loc[lambda x: x>0]

Text              76
Annotation      1105
DateModified      95
BookTitle       1201
dtype: int64

In [11]:
anotaciones_df2.shape

(1201, 11)

In [12]:
anotaciones_df.head(1)

,VolumeID,Text,Annotation,DateCreated,DateModified,ChapterProgress,Type,BookTitle,Chapter,Author,BookmarkID
0,file:///mnt/onboard/.kobo/dropbox/Juana_de_Arc...,dos brazas.\n,None,2024-08-25T14:14:01.000,2024-08-25T14:14:01Z,0.019093,highlight,Juana de Arco,Presentación,None,166dcffc-59f9-465f-880c-480827bf06cd


In [13]:
anotaciones_df2.head(1)

,VolumeID,Text,Annotation,DateCreated,DateModified,ChapterProgress,Type,BookTitle,Chapter,Author,BookmarkID
0,file:///mnt/onboard/.kobo/dropbox/Juana_de_Arc...,dos brazas.\n,None,2024-08-25T14:14:01.000,2024-08-25T14:14:01Z,0.019093,highlight,None,Juana de Arco,Mark Twain,166dcffc-59f9-465f-880c-480827bf06cd


In [14]:
libros_df = db.get_query_df(QUERY_BOOKS)

In [15]:
libros_df.head().VolumeID.values[0]

'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub'

In [16]:
QUERY_CHAPTERS = """
    SELECT DISTINCT
        b.VolumeID,
        c.BookTitle, 
        c.Title,
        c.Attribution as Author
    FROM
        Bookmark b INNER JOIN content c ON b.ContentID = c.ContentID
    ORDER BY
        c.Title;
"""
capitulos_df = db.get_query_df(QUERY_CHAPTERS)

In [20]:
bookmark_df.head(1)

,BookmarkID,VolumeID,ContentID,StartContainerPath,StartContainerChildIndex,EndContainerPath,EndContainerChildIndex,Text,Annotation,ExtraAnnotationData,DateCreated,ChapterProgress,DateModified,Type
0,a5090ecd-389f-4167-b499-d4bb11d75c21,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,0,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,0,None,None,OEBPS/Text/007.xhtml#point(/1/4/278/1:454),2022-03-21T21:03:49.079,0.206573,2022-03-21T21:03:49Z,dogear


In [30]:
content_df.loc[lambda x: x.BookTitle.notna()].loc[lambda x: x.BookTitle.str.contains("12")]\
    .nunique().loc
    # .iloc[0]

ContentID                19
ContentType               1
MimeType                  1
BookID                    1
BookTitle                 1
ImageId                   0
Title                    19
Attribution               0
Description               0
DateCreated               0
adobe_location           19
Publisher                 0
DateLastRead              0
FirstTimeReading          1
ChapterIDBookmarked       0
NumShortcovers            0
VolumeIndex              19
___NumPages               1
ReadStatus                1
___SyncTime               1
___UserID                 1
___FileSize               1
___PercentRead            1
Language                  0
IsDownloaded              1
AverageRating             1
Depth                     1
PageProgressDirection     0
ISBN                      0
EpubType                  1
Series                    0
SeriesNumber              0
Subtitle                  0
WorkId                    0
RatingCount               1
RedirectPreviewUrl  

In [19]:
db.get_query_df("""select * from Bookmark
                 where VolumeID like '%12_regl%' and Text is not Null
                  """)#[['VolumeID', 'ContentID']].VolumeID.unique()

,BookmarkID,VolumeID,ContentID,StartContainerPath,StartContainerChildIndex,StartOffset,EndContainerPath,EndContainerChildIndex,EndOffset,Text,Annotation,ExtraAnnotationData,DateCreated,ChapterProgress,Hidden,Version,DateModified,Creator,UUID,UserID,SyncTime,Published,ContextString,Type
0,c773efbe-0cc0-448a-89f0-c9923c561cfd,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,OEBPS/Text/cap002.xhtml#point(/1/4/193:4),-99,0,OEBPS/Text/cap002.xhtml#point(/1/4/194/1:124),-99,0,Tratarte como a alguien a quien tienes la resp...,None,None,2024-09-06T13:11:16.000,0.191646,false,None,2024-09-06T13:11:16Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
1,ef4053aa-945d-43ed-895a-4a047ead3f80,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,OEBPS/Text/cap004.xhtml#point(/1/4/70/1:1105),-99,0,OEBPS/Text/cap004.xhtml#point(/1/4/70/1:1146),-99,0,lo que te propones determina lo que ves.,None,None,2024-09-07T12:55:08.000,0.272727,false,None,2024-09-07T12:55:08Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
2,82a14832-628c-434d-94db-e10d918aa698,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,OEBPS/Text/cap004.xhtml#point(/1/4/50/1:330),-99,0,OEBPS/Text/cap004.xhtml#point(/1/4/50/1:449),-99,0,Cazar no es otra cosa que establecer un objeti...,None,None,2024-09-07T12:47:53.000,0.262899,false,None,2024-09-07T12:47:53Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
3,4c45fb31-632f-4343-bfcf-3a0e42cc040a,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,OEBPS/Text/cap003.xhtml#point(/1/4/103:4),-99,0,OEBPS/Text/cap003.xhtml#point(/1/4/104/1:118),-99,0,"Si mantengo una relación malsana contigo, quiz...",None,None,2024-09-06T14:19:53.000,0.238329,false,None,2024-09-06T14:19:53Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
4,da6e9109-f9f8-4682-bcb9-d4f767a8623a,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,OEBPS/Text/cap005.xhtml#point(/1/4/172/1:1303),-99,0,OEBPS/Text/cap005.xhtml#point(/1/4/172/1:1438),-99,0,El resentimiento alimenta el deseo de venganza...,None,None,2024-09-08T13:41:29.000,0.388206,false,None,2024-09-08T13:41:29Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
5,39ef85c4-0625-44f0-a52e-9caa2e5513f4,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,OEBPS/Text/cap011.xhtml#point(/1/4/40/1:97),-99,0,OEBPS/Text/cap011.xhtml#point(/1/4/40/1:147),-99,0,"si no puedes entender por qué alguien hizo algo,",None,None,2024-08-29T09:03:24.000,0.729730,false,None,2024-08-29T09:03:24Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
6,8df8c414-c9a5-447a-803d-6cd2555cd46a,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,OEBPS/Text/cap002.xhtml#point(/1/4/86/1:361),-99,0,OEBPS/Text/cap002.xhtml#point(/1/4/86/1:505),-99,0,tienes que tener un pie plantado en lo que ya ...,None,None,2024-09-05T12:51:47.000,0.149877,false,None,2024-09-05T12:51:47Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
7,3d2b13a4-32a1-44ce-b96d-8ca2e95aebdd,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,OEBPS/Text/cap005.xhtml#point(/1/4/124/1:1360),-99,0,OEBPS/Text/cap005.xhtml#point(/1/4/125:0),-99,0,cuando se enfrentan a su primer contacto direc...,None,None,2024-09-08T13:24:18.000,0.361179,false,None,2024-09-08T13:24:18Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
8,e525c770-e447-4857-a4d3-2e4605acf199,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,OEBPS/Text/cap004.xhtml#point(/1/4/144/1:305),-99,0,OEBPS/Text/cap00

In [ ]:
content: 
    ContentID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub#(0)OEBPS/Text/cubierta.xhtml'
    BookID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub'

Bookmark:
    VolumeID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub'
    ContentID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub#(7)OEBPS/Text/cap005.xhtml'


In [33]:
capitulos_df

,VolumeID,BookTitle,Title,Author
0,file:///mnt/onboard/.kobo/dropbox/La_ciudad_de...,La ciudad de los prodigios,1,None
1,file:///mnt/onboard/.kobo/dropbox/Grit_Angela_...,Grit,1 Persevera hasta el final,None
2,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,Surrounded by Idiots,1. Communication Happens on the Listener’s Terms,None
3,file:///mnt/onboard/.kobo/dropbox/Psicologia_o...,Psicología oscura,1. Cómo crear una nueva creencia,None
4,file:///mnt/onboard/.kobo/dropbox/El_poder_de_...,El poder de confiar en ti mismo,1. La base de la confianza en sí mismo,None
...,...,...,...,...
266,file:///mnt/onboard/.kobo/dropbox/La_plata_de_...,La plata de Britania,XIX,None
267,file:///mnt/onboard/.kobo/dropbox/La_plata_de_...,La plata de Britania,XXV,None
268,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,Las 5 trampas del amor,"¿Por qué es tan difícil obtener, mantener y so...",None
269,file:///mnt/onboard/.kobo/dropbox/El_universo_...,El universo de lo sencillo,¿Por qué volamos?,None


In [16]:
anotaciones_df2.loc[lambda x: x.VolumeID.str.contains("12_reglas")].VolumeID.unique()

array(['file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub'],
      dtype=object)

In [17]:
anotaciones_df.loc[lambda x: x.VolumeID.str.contains("12_reglas")].VolumeID.unique()

array(['file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub'],
      dtype=object)

In [25]:
content_df.loc[lambda x: x.ContentID.str.contains("12_reglas")]

,ContentID,ContentType,MimeType,BookID,BookTitle,ImageId,Title,Attribution,Description,DateCreated,ShortCoverKey,adobe_location,Publisher,IsEncrypted,DateLastRead,FirstTimeReading,ChapterIDBookmarked,ParagraphBookmarked,BookmarkWordOffset,NumShortcovers,VolumeIndex,___NumPages,ReadStatus,___SyncTime,___UserID,PublicationId,___FileOffset,___FileSize,___PercentRead,___ExpirationStatus,FavouritesIndex,Accessibility,ContentURL,Language,BookshelfTags,IsDownloaded,FeedbackType,AverageRating,Depth,PageProgressDirection,InWishlist,ISBN,WishlistedDate,FeedbackTypeSynced,IsSocialEnabled,EpubType,Monetization,ExternalId,Series,SeriesNumber,Subtitle,WordCount,Fallback,RestOfBookEstimate,CurrentChapterEstimate,CurrentChapterProgress,PocketStatus,UnsyncedPocketChanges,ImageUrl,DateAdded,WorkId,Properties,RenditionSpread,RatingCount,ReviewsSyncDate,MediaOverlay,MediaOverlayType,RedirectPreviewUrl,PreviewFileSize,EntitlementId,CrossRevisionId,DownloadUrl,ReadStateSynced,TimesStartedReading,TimeSpentReading,LastTimeStartedReading,LastTimeFinishedReading,ApplicableSubscriptions,ExternalIds,PurchaseRevisionId,SeriesID,SeriesNumberFloat,AdobeLoanExpiration,HideFromHomePage,IsInternetArchive,titleKana,subtitleKana,seriesKana,attributionKana,publisherKana,IsPurchaseable,IsSupported,AnnotationsSyncToken,DateModified,StorePages,StoreWordCount,StoreTimeToReadLowerEstimate,StoreTimeToReadUpperEstimate,Duration,IsAbridged,SyncConflictType
1167,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,12 reglas para vivir,None,Cubierta,None,None,None,None,OEBPS/Text/cubierta.xhtml,None,false,None,true,None,0,0,NaN,0,-1,0,2022-12-10T10:56:27Z,adobe_user,None,0.0,0,0,NaN,-1,-1,None,None,None,1,0,0.0,1,None,FALSE,None,None,0,true,-1,2,None,None,None,None,-1,None,NaN,NaN,NaN,0,None,None,None,None,None,None,0,None,None,None,None,NaN,None,None,None,false,NaN,NaN,None,None,None,None,None,None,NaN,None,None,FALSE,None,None,None,None,None,true,true,None,None,0,0,0,0,0,None,0
1168,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,12 reglas para vivir,None,12 reglas para vivir,None,None,None,None,OEBPS/Text/titulo.xhtml,None,false,None,true,None,0,0,NaN,1,-1,0,2022-12-10T10:56:27Z,adobe_user,None,0.0,0,0,NaN,-1,-1,None,None,None,1,0,0.0,1,None,FALSE,None,None,0,true,-1,2,None,None,None,None,-1,None,NaN,NaN,NaN,0,None,None,None,None,None,None,0,None,None,None,None,NaN,None,None,None,false,NaN,NaN,None,None,None,None,None,None,NaN,None,None,FALSE,None,None,None,None,None,true,true,None,None,0,0,0,0,0,None,0
1169,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,12 reglas para vivir,None,Nota preliminar,None,None,None,None,OEBPS/Text/cap000.xhtml,None,false,None,true,None,0,0,NaN,2,-1,0,2022-12-10T10:56:27Z,adobe_user,None,0.0,0,0,NaN,-1,-1,None,None,None,1,0,0.0,1,None,FALSE,None,None,0,true,-1,2,None,None,None,None,-1,None,NaN,NaN,NaN,0,None,None,None,None,None,None,0,None,None,None,None,NaN,None,None,None,false,NaN,NaN,None,None,None,None,None,None,NaN,None,None,FALSE,None,None,None,None,None,true,true,None,None,0,0,0,0,0,None,0
1170,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,9,application/epub+zip,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,12 reglas para vivir,None,REGLA 1. Enderézate y mantén los hombros hacia...,None,None,None,None,OEBPS/Text/cap001.xhtml,None,false,None,true,None,0,0,NaN,3,-1,0,2022-12-10T10:56:27Z,adobe_user,None,0.0,0,0,NaN,-1,-1,None,None,None,1,0,0.0,1,None,FALSE,None,None,0,true,-1,2,None,None,None,None,-1,None,NaN,NaN,NaN,0,None,None,None,None,None,None,0,None,None,None,None,NaN,None,None,None,false,NaN,NaN,None,None,None,None,None,None,NaN,None,None,FALSE,None,None,None,None,None,true,true,None,None,0,0,0,0,0,None,0
1171,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,9,application/epub+zip,file:///mnt/onboard/.kob

In [27]:
bookmark_df

,BookmarkID,VolumeID,ContentID,StartContainerPath,StartContainerChildIndex,StartOffset,EndContainerPath,EndContainerChildIndex,EndOffset,Text,Annotation,ExtraAnnotationData,DateCreated,ChapterProgress,Hidden,Version,DateModified,Creator,UUID,UserID,SyncTime,Published,ContextString,Type
0,a5090ecd-389f-4167-b499-d4bb11d75c21,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,0,0,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,0,0,None,None,OEBPS/Text/007.xhtml#point(/1/4/278/1:454),2022-03-21T21:03:49.079,0.206573,false,None,2022-03-21T21:03:49Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear
1,d70ed735-e49b-4097-a5ea-29d87ba55d4c,file:///mnt/onboard/.kobo/dropbox/Loba_Negra_J...,file:///mnt/onboard/.kobo/dropbox/Loba_Negra_J...,file:///mnt/onboard/.kobo/dropbox/Loba_Negra_J...,0,0,file:///mnt/onboard/.kobo/dropbox/Loba_Negra_J...,0,0,None,None,OEBPS/Text/Intro.xhtml#point(/1/4/4/22/1:257),2022-08-14T16:02:13.741,0.288770,false,None,2022-08-14T16:02:13Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear
2,ab6d7a39-6ae0-4073-84d3-536c1a32a1be,file:///mnt/onboard/.kobo/dropbox/Los_demonios...,file:///mnt/onboard/.kobo/dropbox/Los_demonios...,file:///mnt/onboard/.kobo/dropbox/Los_demonios...,0,0,file:///mnt/onboard/.kobo/dropbox/Los_demonios...,0,0,None,None,OEBPS/Text/part0041.xhtml#point(/1/4/46/1:0),2022-08-16T16:11:46.015,0.450575,false,None,2022-08-16T16:11:46Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear
3,e44bc853-5125-4441-bb62-1855ba97b424,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,OEBPS/Text/Sec0208.xhtml#point(/1/4/28/1:186),0,0,OEBPS/Text/Sec0208.xhtml#point(/1/4/28/1:186),0,0,None,None,       S t a r t K e y             S c r e ...,2023-01-13T19:08:37.864,0.453988,false,None,2023-01-13T19:08:37Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,markup
4,1734a906-4785-4ef4-9880-a8d4350e1b2e,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,0,0,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,0,0,None,None,OEBPS/Text/Sec0202.xhtml#point(/1/4/30/1:0),2023-01-10T23:04:12.015,0.226994,false,None,2023-01-10T23:04:12Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,1fceb49e-4547-4e2d-b755-a6c36e4de1b9,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,OEBPS/part0007.xhtml#point(/1/4/2/104/2/1:228),-99,0,OEBPS/part0007.xhtml#point(/1/4/2/104/2/1:276),-99,0,entiende su propia violencia como mera respuesta,None,None,2025-01-11T18:03:13.000,0.236994,false,None,2025-01-11T18:03:13Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
1197,b64eca6b-4773-4ff5-80d2-7cb0c18bd2b4,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,OEBPS/part0006.xhtml#point(/1/4/2/164/2/1:68),-99,0,OEBPS/part0006.xhtml#point(/1/4/2/165:0),-99,0,"el narcisismo no es un exceso, sino un déficit...",None,None,2025-01-08T21:52:41.000,0.156069,false,None,2025-01-08T21:52:41Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
1198,2d676f06-1e74-4000-a029-fee568d44370,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,OEBPS/part0006.xhtml#point(/1/4/2/80/2/1:100),-99,0,OEBPS/part0006.xhtml#point(/1/4/2/81:0),-99,0,desde las primeras horas después de nacer pose...,None,None,2025-01-06T23:45:12.000,0.132948,false,None,2025-01-06T23:45:12Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
1199,5460a42e-1b60-4c53-b683-acc62f14ae5f,file:///mnt/onboard/.kobo/dropbox/Amanecer_Con...,file:///mnt/onboa

## Exploratorio de Todas las Tablas

In [5]:
db_path = "KoboReader.sqlite"  # Ruta a tu archivo SQLite
db = SQLiteWrapper(db_path)
db.connect()  # Establecer conexión

In [71]:
tables_info_df = db.get_tables_info() 

Tablas interesantes:
* content: contenido interesante
* Event
* Bookmark

Otras tablas:
* Shelf: Colecciones de libros. Por ejemplo, Histórica, Thriller, Filosofía...
* ShelfContent: Contenidos de las colecciones de Shelf
* WordList: Lista de palabras guardadas
* DropboxItem: Contenidos de dropbox
* user: Información del usuario. Revisar
* content_settings: configuración de lectura de varios libros
* Achievement: Premios de lectura

In [59]:
tables_info_df.sort_values("num_filas")

,nombre,num_filas,num_cols,col_names
0,DbVersion,1,1,[version]
14,user,1,29,"[UserID, UserKey, UserDisplayName, UserEmail, ..."
22,Tab,1,14,"[tabId, tabType, browseTabType, displayTitle, ..."
30,KoboPlusAssetGroup,1,8,"[Id, AssetGroup, Timestamp, Name, Url, Etag, T..."
32,GDriveItem,1,3,"[Id, Json, Path]"
13,Shelf,2,11,"[CreationDate, Id, InternalName, LastModified,..."
17,AnalyticsEvents,4,8,"[Id, Type, Timestamp, Attributes, Metrics, Tes..."
28,SubscriptionProducts,4,7,"[CrossRevisionId, Id, Name, IsPreOrder, Tiers,..."
12,content_settings,20,19,"[ContentID, ContentType, DateModified, Reading..."
9,volume_tabs,22,2,"[volumeId, tabId]"


In [24]:
db.get_query_df("select * from content limit 10")

,ContentID,ContentType,MimeType,BookID,BookTitle,ImageId,Title,Attribution,Description,DateCreated,ShortCoverKey,adobe_location,Publisher,IsEncrypted,DateLastRead,FirstTimeReading,ChapterIDBookmarked,ParagraphBookmarked,BookmarkWordOffset,NumShortcovers,VolumeIndex,___NumPages,ReadStatus,___SyncTime,___UserID,PublicationId,___FileOffset,___FileSize,___PercentRead,___ExpirationStatus,FavouritesIndex,Accessibility,ContentURL,Language,BookshelfTags,IsDownloaded,FeedbackType,AverageRating,Depth,PageProgressDirection,InWishlist,ISBN,WishlistedDate,FeedbackTypeSynced,IsSocialEnabled,EpubType,Monetization,ExternalId,Series,SeriesNumber,Subtitle,WordCount,Fallback,RestOfBookEstimate,CurrentChapterEstimate,CurrentChapterProgress,PocketStatus,UnsyncedPocketChanges,ImageUrl,DateAdded,WorkId,Properties,RenditionSpread,RatingCount,ReviewsSyncDate,MediaOverlay,MediaOverlayType,RedirectPreviewUrl,PreviewFileSize,EntitlementId,CrossRevisionId,DownloadUrl,ReadStateSynced,TimesStartedReading,TimeSpentReading,LastTimeStartedReading,LastTimeFinishedReading,ApplicableSubscriptions,ExternalIds,PurchaseRevisionId,SeriesID,SeriesNumberFloat,AdobeLoanExpiration,HideFromHomePage,IsInternetArchive,titleKana,subtitleKana,seriesKana,attributionKana,publisherKana,IsPurchaseable,IsSupported,AnnotationsSyncToken,DateModified,StorePages,StoreWordCount,StoreTimeToReadLowerEstimate,StoreTimeToReadUpperEstimate,Duration,IsAbridged,SyncConflictType
0,22b09ad5-4b26-4130-9424-9cee56aea26f,6,application/x-kobo-epub+zip,None,None,68fd5c91-e759-4154-9b61-eef242623461,Something to Hide,Elizabeth George,<p><strong>Detective Sergeant Barbara Havers a...,2022-01-11T05:00:00.0000000Z,None,None,Penguin Publishing Group,false,None,true,None,0,0,0.0,0,-1,0,2022-01-15T18:53:29Z,,None,NaN,0,0,0.0,-1,-1,None,en,None,false,0,5,0,default,FALSE,9780593296851,None,0,true,13,2,None,A Lynley Novel,None,A Lynley Novel,-1,None,0.0,0.0,0.0,0,,None,None,d16d6eff-b841-3d04-ab2b-13b5158458e6,None,None,1,None,None,None,true,1004170.0,,04710a6e-cb96-3ee2-98d5-d11701f288bd,false,true,0.0,0.0,None,None,None,None,None,None,0.0,None,false,false,None,None,None,None,None,true,true,None,None,0,0,0,0,0,None,0
1,300e8011-45e8-45c8-a010-917c21a983d0,6,application/x-kobo-epub+zip,None,None,1f968fbd-c98a-4880-a02d-aa9cd1f811c6,End of Days,Brad Taylor,<p><strong>Pike Logan must stop a deranged kil...,2022-01-11T05:00:00.0000000Z,None,None,HarperCollins,false,None,true,None,0,0,0.0,1,-1,0,2022-01-15T18:53:29Z,,None,NaN,0,0,0.0,-1,-1,None,en,None,false,0,4,0,default,FALSE,9780062886125,None,0,true,13,2,None,Pike Logan,16,A Pike Logan Novel,-1,None,0.0,0.0,0.0,0,,None,None,b8b4d66b-0179-356e-bb21-70c7aa6d2e29,None,None,1,None,None,None,true,871774.0,,33549b17-771a-326e-b19d-67520108fa36,false,true,0.0,0.0,None,None,None,None,None,None,16.0,None,false,false,None,None,None,None,None,true,true,None,None,0,0,0,0,0,None,0
2,6963c490-b680-4e56-aeca-32d8c0dbd50e,6,application/x-kobo-epub+zip,None,None,0c749bd4-38ac-41c5-9d86-0c162c09e4f4,Wahala,Nikki May,"<p><strong>""Utterly winning…brings to mind Sex...",2022-01-11T05:00:00.0000000Z,None,None,HarperCollins,false,None,true,None,0,0,0.0,2,-1,0,2022-01-15T18:53:29Z,,None,NaN,0,0,0.0,-1,-1,None,en,None,false,0,0,0,default,FALSE,9780063084261,None,0,true,13,2,None,None,None,A Novel,-1,None,0.0,0.0,0.0,0,,None,None,4b787b13-ab87-4425-9cf3-0b348a32074b,None,None,0,None,None,None,true,769141.0,,17d18239-7275-3c4f-a98a-05a02328d001,false,true,0.0,0.0,None,None,None,None,None,None,0.0,None,false,false,None,None,None,None,None,true,true,None,None,0,0,0,0,0,None,0
3,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Sinopsis,None,None,None,None,OEBPS/Text/part0002.html,None,false,None,true,None,0,0,NaN,0,-1,0,2022-01-15T19:28:05Z,adobe_user,None,0.0,0,0,NaN,-1,-1,None,None,None,1,0,0,1,None,FALSE,None,None,0,true,-1,2,None,None,None,None,-1,None,NaN,NaN,NaN,0,None,None,None,None,None,N

In [58]:
db.get_query_df("select * from DbVersion")

,version
0,174


In [ ]:
db.get_query_df("select * from volume_shortcovers limit 10 ")

,volumeId,shortcoverId,VolumeIndex
0,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,0
1,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,1
2,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,2
3,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,3
4,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,4
5,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,5
6,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,6
7,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,7
8,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,8
9,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9


In [13]:
db.get_query_df("select * from user")

,UserID,UserKey,UserDisplayName,UserEmail,___DeviceID,FacebookAuthToken,HasMadePurchase,IsOneStoreAccount,IsChildAccount,RefreshToken,...,Storefront,NewUserPromoCurrency,NewUserPromoValue,KoboAccessToken,KoboAccessTokenExpiry,AnnotationsSyncToken,PrivacyPermissions,AnnotationsMigrated,NotebookSyncTime,NotebookSyncToken
0,83b63413-c6de-45c8-a874-7cadf24716d7,beb19512-af71-45d5-bcb4-fe1ef804aa40,jaguirrepeman@gmail.com,jaguirrepeman@gmail.com,5a9f4f0bf0462fcbefa84fc16badbc1508283270611d49...,None,false,true,false,1R38nbKzczPnuQvyq2yRI25esfR4gsrKvPFO0zVSKoE7hm...,...,ES,None,0.0,eyJhbGciOiJSUzI1NiIsImtpZCI6IjA4NkI4NjkzNUQyQ0...,2025-01-15T08:17:21Z,None,    ,true,2025-01-12T22:02:11.0Z,None


In [14]:
db.get_query_df("select * from Tab")

,tabId,tabType,browseTabType,displayTitle,parentTabId,isDefault,maxSize,totalResults,updateFrequencyMin,imageID,isLeaf,hasFeaturedLists,etag,language
0,SomeFakeWishlistTabID,2,0,None,None,false,0,0,0,None,false,false,"W/""Tm9Db250ZW50""",None


In [15]:
db.get_query_df("select * from KoboPlusAssetGroup")

,Id,AssetGroup,Timestamp,Name,Url,Etag,TimestampTo,Shown
0,77c91545-d80f-46be-86d9-357079015fb2,EPD-KoboPlus-ReadOnly-NeverSubscribed,2021-10-01T00:00:00.0000000Z,EPD-KoboPlus-ReadOnly-NeverSubscribed,https://www.kobo.com/{region}/{language}/plus/...,W/NjM4NTU1MjYxNDkwMDAwMDAwLTU5MDMw,None,true


In [16]:
db.get_query_df("select * from GDriveItem")

,Id,Json,Path
0,10tC86zWmDZ7Inl-Wy26Cj-iSnMkolBVJ,"{\n ""contentHints"": {\n ""thumbnail"":...",/Cómo añadir libros a tu eReader usando Google...


In [17]:
db.get_query_df("select * from Shelf")

,CreationDate,Id,InternalName,LastModified,Name,Type,_IsDeleted,_IsVisible,_IsSynced,_SyncTime,LastAccessed
0,2024-09-03T22:29:45Z,7a58f07d-36d5-45ac-8a05-b8bc873f130c,Filosofía y autoayuda,2024-09-03T22:29:45Z,Filosofía y autoayuda,UserTag,false,true,true,2024-09-05T09:44:53Z,2024-12-29T14:49:42.255
1,2024-09-03T22:29:45Z,79d8a4e4-2ced-40a0-866b-ba46cfd196b0,Histórica,2024-09-03T22:29:45Z,Histórica,UserTag,false,true,true,2024-09-05T09:44:53Z,2025-01-10T18:24:02.220


In [18]:
db.get_query_df("select * from AnalyticsEvents")

,Id,Type,Timestamp,Attributes,Metrics,TestGroups,ClientApplicationVersion,Mandatory
0,c9748c97-fbcd-41f8-bd2e-41d9c8d04e30,TimeToUpdate,2022-01-15T18:49:04Z,{},"{""time"": 109}",{},4.29.18480,false
1,b80ebed0-14c7-4e3a-9f02-0478008ffafb,AppStart,2022-01-15T18:49:05Z,{},{},{},4.29.18480,false
2,1c69fa15-1285-4a76-8e78-30e021417c4b,WelcomeScreen,2022-01-15T18:49:14Z,"{""CurrentLocale"": ""es_ES""}",{},{},4.29.18480,false
3,1cea63da-fb7b-41a7-b7f4-05e7cd0db7a8,WifiSetup,2022-01-15T18:49:17Z,{},{},{},4.29.18480,false


In [10]:
db.get_query_df("select * from SubscriptionProducts")

,CrossRevisionId,Id,Name,IsPreOrder,Tiers,ActivationDate,DeactivationDate
0,be9e9ac0-5823-4047-8281-069986c626b9,a532a22f-fac5-45a8-a7eb-2dcc596eaadc,Kobo Plus Read,false,               P h a s e s   \t          ...,2024-07-03T09:00:00.0000000Z,None
1,c382502f-4df7-4925-a69b-f85628f6814f,b705254f-4b90-4664-bea6-d091291827b9,Kobo Plus Prorated Upgrade,false,               P h a s e s   \t          ...,2024-07-03T09:00:00.0000000Z,None
2,416bfce4-e4ae-4730-8314-5857dece58a6,27ef4486-eed0-4bbc-ac55-ded89007f68e,Kobo Plus Read & Listen,false,               P h a s e s   \t          ...,2024-07-03T09:00:00.0000000Z,None
3,93dd7c18-51f1-493f-bb5f-d9e62b974bee,1ecb8932-4938-4d65-a623-e94c28609513,Kobo Plus Listen,false,               P h a s e s   \t          ...,2024-07-03T09:00:00.0000000Z,None


In [21]:
db.get_query_df("select * from content_settings limit 5")

,ContentID,ContentType,DateModified,ReadingFontFamily,ReadingFontSize,ReadingAlignment,ReadingLineHeight,ReadingLeftMargin,ReadingRightMargin,ReadingPublisherMode,ActivityFacebookShare,RecentBookSearches,AuthorNotesShown,LastAuthorNotesSyncTime,ZoomFactor,BTBFooterSection,SelectedDictionary,StillReading,SeriesShown
0,file:///mnt/onboard/La_bestia_Carmen_Mola.epub,6,2022-01-15T19:35:15.627,default,28,None,1.35,2,2,None,TRUE,None,false,None,1,None,None,false,false
1,file:///mnt/onboard/.kobo/dropbox/Soy_introver...,6,2022-02-16T20:23:02.739,default,28,None,1.20,2,2,None,TRUE,None,false,None,1,None,None,false,false
2,file:///mnt/onboard/.kobo/dropbox/Los_demonios...,6,2022-08-18T09:27:37.577,default,28,,1.20,2,2,None,TRUE,None,false,None,1,None,None,false,false
3,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,6,2023-01-13T18:51:18.428,default,30,,1.20,2,2,None,TRUE,None,false,None,1,None,None,false,false
4,file:///mnt/onboard/.kobo/dropbox/Diez_negrito...,6,2023-02-13T00:36:21.842,default,28,None,1.20,2,2,None,TRUE,None,false,None,1,None,None,false,false


In [30]:
db.get_query_df("select * from Achievement")\
        .loc[lambda x: x.PercentComplete= 100].CompleteDescription.values

array(["You're now part of Kobo, and an official Booklover!",
       'You started reading a new book.',
       'You highlighted your first passage.',
       "That's 10 new definitions for you. Now, maybe it's time to go outside?",
       'Make eReading more fun, social and meaningful \\n by building your Reading Life with Kobo. \\n \\n You can track your reading life with interesting \\n statistics and celebrate reading milestones with \\n fun and surprising awards.',
       "You've mastered the art of the long press - the best way to manage your library and dive into the details of a book.",
       'You\'ve finished a book. Is it true that "the good ended happily, and the bad unhappily"?(Thanks to Oscar Wilde for the quote!)',
       "Clearly you couldn't put that book down! You read for two hours straight!",
       "Go big or go home! You've turned 10,000 pages with Kobo."],
      dtype=object)

In [35]:
db.get_query_df("select * from Rules limit 5")

,AchievementId,EventProperty,EventType,GoalValue,Id,Operation,ParentRuleId,ConjunctionType,IsConjunction,Checksum
0,00000000-0000-0000-0000-000000001002,EventCount,27,1,49,1,None,0,0,256e75484f5aad07cb9c9e84b21d0040
1,00000000-0000-0000-0000-000000001008,EventCount,6,1,50,1,None,0,0,b770d31f96b6c145e1902fce4aef671e
2,00000000-0000-0000-0000-00000000100D,EventCount,28,10,51,1,None,0,0,ae35306e2c666c42a76d23acb8a8c48f
3,00000000-0000-0000-0000-000000001000,EventCount,0,1,0,1,None,0,false,b78637ae9596314835a5d2178f92833b
4,00000000-0000-0000-0000-000000001001,EventCount,"35,36",0,1,999,None,1,true,3c707365979a0bba64c589698c77a07e


In [34]:
db.get_query_df("select * from ShelfContent limit 5")

,ShelfName,ContentId,DateModified,_IsDeleted,_IsSynced
0,Filosofía y autoayuda,file:///mnt/onboard/.kobo/dropbox/Mapas_de_sen...,2024-09-02T17:09:39Z,false,false
1,Filosofía y autoayuda,file:///mnt/onboard/.kobo/dropbox/Manipulacion...,2024-09-02T17:09:39Z,false,false
2,Filosofía y autoayuda,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,2024-09-02T17:09:39Z,false,false
3,Filosofía y autoayuda,file:///mnt/onboard/.kobo/dropbox/El_Principe_...,2024-09-02T17:09:39Z,false,false
4,Filosofía y autoayuda,file:///mnt/onboard/.kobo/dropbox/30_dias_Camb...,2024-09-02T17:09:39Z,false,false


In [37]:
db.get_query_df("select * from WordList ")

,Text,VolumeId,DictSuffix,DateCreated
0,envarados,file:///mnt/onboard/.kobo/dropbox/Nueva_York_E...,-es,2024-08-20T20:01:28Z
1,argüir,file:///mnt/onboard/.kobo/dropbox/Nueva_York_E...,-es,2024-08-20T21:45:24Z
2,astroso,file:///mnt/onboard/.kobo/dropbox/Ivanhoe_Walt...,-es,2024-08-21T15:11:33Z
3,frívolo,file:///mnt/onboard/.kobo/dropbox/Ivanhoe_Walt...,-es,2024-08-21T15:41:17Z
4,"escabel,",file:///mnt/onboard/.kobo/dropbox/Ivanhoe_Walt...,-es,2024-08-21T15:58:31Z
...,...,...,...,...
78,spiel.,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,-en-es,2025-01-06T08:24:31Z
79,fudge,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,-en-es,2025-01-06T08:26:14Z
80,stifle,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,-en-es,2025-01-08T22:17:02Z
81,stonemason,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,-en-es,2025-01-10T17:56:24Z


In [20]:
db.get_query_df("select * from Activity ")

,Id,Enabled,Type,Action,Date,Data
0,7eb527d6-21bb-407c-bf00-47241e687d32,true,Bookstore,2,2022-01-15T20:22:47,      $ B O O K S T O R E _ T A B _ T Y P E  ...
1,None,true,Library,1,2022-01-15T20:28:09,"      , L I B R A R Y _ A C T I V I T Y _ R E..."
2,None,false,WhatsNew,2,2022-02-02T23:43:51,        W H A T S _ N E W _ D A T A   \n   \r...
3,file:///mnt/onboard/.kobo/dropbox/Scar_Tissue_...,true,RecentBook,2,2022-02-19T20:47:04,
4,file:///mnt/onboard/.kobo/dropbox/Soy_introver...,true,RecentBook,2,2022-02-26T15:24:37,
...,...,...,...,...,...,...
113,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,true,RecentBook,2,2025-01-10T19:24:01,      6 A N N O T A T I O N _ S T A T S _ N O...
114,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,true,RecentBook,2,2025-01-12T17:47:02,      6 A N N O T A T I O N _ S T A T S _ N O...
115,7eb527d6-21bb-407c-bf00-47241e687d32,false,TopPicksTab,1,2025-01-12T23:03:40,      0 D I S C O V E R Y _ V O L U M E _ I D...
116,file:///mnt/onboard/.kobo/dropbox/Amanecer_Con...,true,RecentBook,2,2025-01-14T20:04:18,      6 A N N O T A T I O N _ S T A T S _ N O...


In [42]:
db.get_query_df("select * from DropboxItem").Json[0]

'{\n    "client_modified": "2022-01-30T22:08:16Z",\n    "content_hash": "d8a4640dd72325d32a383da2d0f4d7098bf8ac306851a638dea25c991ebb7e8d",\n    "has_explicit_shared_members": false,\n    "id": "id:3e6PcycqSwUAAAAAAAAAHQ",\n    "media_info": "pending",\n    "name": "Soy_introvertido_Y_que_Steve_Allen.epub",\n    "path_display": "/Soy_introvertido_Y_que_Steve_Allen.epub",\n    "path_lower": "/soy_introvertido_y_que_steve_allen.epub",\n    "property_groups": [\n    ],\n    "rev": "5d6d3e7b0d0b38c86dc51",\n    "server_modified": "2022-01-30T22:08:17Z",\n    "sharing_info": {\n        "read_only": false\n    },\n    "size": "231800"\n}\n'

In [47]:
db.get_query_df("select * from KoboPlusAssets where Language = 'es' ").Value.values

array(['Busca títulos con la etiqueta Kobo Plus en nuestro catálogo.',
       'Añade los eBooks o audiolibros que quieras a tus libros.',
       'Sincroniza tu eReader o aplicación para ver tus libros de Kobo Plus.',
       'Escucha tus audiolibros de Kobo Plus con la aplicación Kobo gratuita para iOS o Android.',
       'Comenzar prueba gratuita',
       'Ideal para descubrir nuevos favoritos y explorar títulos de todos los géneros.',
       '¿Cómo funciona?',
       'https://static.kobo.com/assets/KoboPlusMulti_EPD_CA.svg',
       'Disfruta leyendo o escuchando todo lo que quieras con nuestros planes a partir de 8,99  Є al mes.',
       'Visita kobo.com/plus para más información. Descarga la aplicación gratuita de Kobo en kobo.com/apps.',
       'Comenzar prueba gratuita',
       'Lee y escucha todo lo que quieras con nuestras suscripciones a partir de tan solo 8,99  Є al mes. Elige la que más te convenga.',
       'Más información',
       'https://static.kobo.com/assets/KoboPlusMul

In [60]:
db.get_query_df("select * from Event limit 10 ")

Error ejecutando la consulta: Could not decode to UTF-8 column 'ExtraData' with text ''


OperationalError: Could not decode to UTF-8 column 'ExtraData' with text ''

In [50]:
db.get_query_df("select * from Bookmark limit 10 ")

OperationalError: Could not decode to UTF-8 column 'ExtraAnnotationData' with text ''

In [51]:
db.get_query_df("select * from volume_shortcovers limit 10 ")

,volumeId,shortcoverId,VolumeIndex
0,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,0
1,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,1
2,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,2
3,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,3
4,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,4
5,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,5
6,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,6
7,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,7
8,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,8
9,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9


In [56]:
db.get_query_df("""select *
                from content limit 10
                """)

,ContentID,ContentType,MimeType,BookID,BookTitle,ImageId,Title,Attribution,Description,DateCreated,...,IsSupported,AnnotationsSyncToken,DateModified,StorePages,StoreWordCount,StoreTimeToReadLowerEstimate,StoreTimeToReadUpperEstimate,Duration,IsAbridged,SyncConflictType
0,22b09ad5-4b26-4130-9424-9cee56aea26f,6,application/x-kobo-epub+zip,None,None,68fd5c91-e759-4154-9b61-eef242623461,Something to Hide,Elizabeth George,<p><strong>Detective Sergeant Barbara Havers a...,2022-01-11T05:00:00.0000000Z,...,true,None,None,0,0,0,0,0,None,0
1,300e8011-45e8-45c8-a010-917c21a983d0,6,application/x-kobo-epub+zip,None,None,1f968fbd-c98a-4880-a02d-aa9cd1f811c6,End of Days,Brad Taylor,<p><strong>Pike Logan must stop a deranged kil...,2022-01-11T05:00:00.0000000Z,...,true,None,None,0,0,0,0,0,None,0
2,6963c490-b680-4e56-aeca-32d8c0dbd50e,6,application/x-kobo-epub+zip,None,None,0c749bd4-38ac-41c5-9d86-0c162c09e4f4,Wahala,Nikki May,"<p><strong>""Utterly winning…brings to mind Sex...",2022-01-11T05:00:00.0000000Z,...,true,None,None,0,0,0,0,0,None,0
3,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Sinopsis,None,None,None,...,true,None,None,0,0,0,0,0,None,0
4,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Portadilla,None,None,None,...,true,None,None,0,0,0,0,0,None,0
5,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,La expresión inglesa...,None,None,None,...,true,None,None,0,0,0,0,0,None,0
6,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Dedicatoria,None,None,None,...,true,None,None,0,0,0,0,0,None,0
7,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Mapa,None,None,None,...,true,None,None,0,0,0,0,0,None,0
8,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Cita,None,None,None,...,true,None,None,0,0,0,0,0,None,0
9,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,"Cubierta de sangre, la chica se adentra...",None,None,None,...,true,None,None,0,0,0,0,0,None,0


In [55]:
db.get_query_df("""select BookID, BookTitle, count(*)
                from content 
                group by 1,2
                order by 3 desc""")

,BookID,BookTitle,count(*)
0,file:///mnt/onboard/.kobo/dropbox/La_grieta_de...,La grieta del silencio,210
1,None,None,180
2,file:///mnt/onboard/.kobo/dropbox/Google Cloud...,Google Cloud Certified Professional Cloud Arch...,127
3,file:///mnt/onboard/.kobo/dropbox/Amor_zero_In...,Amor zero,118
4,file:///mnt/onboard/.kobo/dropbox/En_busca_de_...,En busca de Klingsor,116
...,...,...,...
154,file:///mnt/onboard/.kobo/dropbox/Novela histó...,CAPÍTULO 19,1
155,file:///mnt/onboard/.kobo/dropbox/Novela histó...,CAPÍTULO 20,1
156,file:///mnt/onboard/.kobo/dropbox/Novela histó...,CAPÍTULO 21,1
157,file:///mnt/onboard/.kobo/dropbox/Thing Explai...,Thing Explainer,1


In [11]:
tables_info_df.sort_values("num_filas")

,nombre,num_filas,num_cols,col_names
0,DbVersion,1,1,[version]
14,user,1,29,"[UserID, UserKey, UserDisplayName, UserEmail, ..."
22,Tab,1,14,"[tabId, tabType, browseTabType, displayTitle, ..."
30,KoboPlusAssetGroup,1,8,"[Id, AssetGroup, Timestamp, Name, Url, Etag, T..."
32,GDriveItem,1,3,"[Id, Json, Path]"
13,Shelf,2,11,"[CreationDate, Id, InternalName, LastModified,..."
17,AnalyticsEvents,4,8,"[Id, Type, Timestamp, Attributes, Metrics, Tes..."
28,SubscriptionProducts,4,7,"[CrossRevisionId, Id, Name, IsPreOrder, Tiers,..."
12,content_settings,20,19,"[ContentID, ContentType, DateModified, Reading..."
9,volume_tabs,22,2,"[volumeId, tabId]"
